In [4]:
#Group No: 4 & Section A 
#DCPP Assignment

from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

def get_page(url):
    # Headersetup for simulation 
    # Simulation the request from browser 
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Connection': 'close',
        'DNT': '1', 
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index 
    df = df.transpose() # Transpose the DataFrame
    
    # Rename the column to "Date"
    
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] 

    for column_index in range(1, len(df.columns)): 
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Removing thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float
        
    return df

def scrape_table(url):
    # Fetch the page 
    page = get_page(url);

    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Yahoo blocked the site 
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

In [5]:
def scrape(symbol):
    print('Attempting to scrape data for ' + symbol)

    df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
    df_balance_sheet = df_balance_sheet.set_index('Date')

    df_income_statement = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol)
    df_income_statement = df_income_statement.set_index('Date')
    
    df_cash_flow = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol)
    df_cash_flow = df_cash_flow.set_index('Date')
    
    df_joined = df_balance_sheet \
        .join(df_income_statement, on='Date', how='outer', rsuffix=' - Income Statement') \
        .join(df_cash_flow, on='Date', how='outer', rsuffix=' - Cash Flow') \
        .dropna(axis=1, how='all') \
        .reset_index()
            
    df_joined.insert(1, 'Symbol', symbol)
    
    return df_joined

In [8]:
def scrape_multi(symbols):
    return pd.concat([scrape(symbol) for symbol in symbols], sort=False)

In [7]:
#Code for extracting sample symbols 
symbols = ['ABB.BO',	'AEGISLOG.BO',	'AMARAJABAT.BO',	'AMBALALSA.BO',	'HDFC.BO',	'ANDHRAPET.BO',	'ANSALAPI.BO']
df_combined = scrape_multi(symbols)
#Complete list of symbols to be pasted from BSE Stock list base file shared seperately  

Attempting to scrape data for ABB.BO
Attempting to scrape data for AEGISLOG.BO
Attempting to scrape data for AMARAJABAT.BO
Attempting to scrape data for AMBALALSA.BO
Attempting to scrape data for HDFC.BO
Attempting to scrape data for ANDHRAPET.BO
Attempting to scrape data for ANSALAPI.BO


In [5]:
df_combined 

,index,Symbol,Date,Total Assets,Total Liabilities Net Minority Interest,Total Equity Gross Minority Interest,Total Capitalization,Common Stock Equity,Capital Lease Obligations,Net Tangible Assets,...,Capital Expenditure,Repayment of Debt,Free Cash Flow,Net Debt,Issuance of Capital Stock,Issuance of Debt,Income from Associates & Other Participating Interests,Special Income Charges,Repurchase of Capital Stock,Other Cash Adjustment Inside Change in Cash
0,12/31/2020,ABB.BO,12/31/2020,7.590500e+07,3.984120e+07,3.606380e+07,3.606380e+07,3.606380e+07,582400.0,3.586630e+07,...,-1213800.0,NaN,1968400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12/31/2019,ABB.BO,12/31/2019,7.700830e+07,4.180720e+07,3.520110e+07,3.520110e+07,3.520110e+07,135400.0,3.498220e+07,...,-1374100.0,0.000000e+00,5294600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12/31/2018,ABB.BO,12/31/2018,9.076900e+07,5.069580e+07,4.007320e+07,4.007320e+07,4.007320e+07,69200.0,3.985480e+07,...,-2436800.0,-6.000000e+06,3817800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12/31/2017,ABB.BO,12/31/2017,8.888050e+07,5.281160e+07,3.606890e+07,3.606890e+07,3.606890e+07,83500.0,3.530830e+07,...,-1858400.0,-4.000000e+02,6139900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,ABB.BO,ttm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1200400.0,NaN,10229600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3/31/2021,AEGISLOG.BO,3/31/2021,3.082152e+07,1.036654e+07,2.045498e+07,2.048794e+07,1.936474e+07,3177613.0,1.934620e+07,...,-3587781.0,-1.255140e+05,827737.0,1.196170e+06,11333.0,1.758976e+06,NaN,NaN,NaN,NaN
1,3/31/2020,AEGISLOG.BO,3/31/2020,2.920702e+07,1.175486e+07,1.745215e+07,1.703113e+07,1.654613e+07,3165349.0,1.652497e+07,...,-1708000.0,-8.274930e+05,-244888.0,3.062050e+05,5667.0,6.000000e+05,NaN,NaN,NaN,NaN
2,3/31/2019,AEGISLOG.BO,3/31/2019,2.428680e+07,9.625903e+06,1.466089e+07,1.447847e+07,1.391277e+07,NaN,1.389028e+07,...,-1540857.0,-1.979820e+05,4022095.0,NaN,0.0,2.523440e+05,NaN,NaN,NaN,NaN
3,3/31/2018,AEGISLOG.BO,3/31/2018,2.213934e+07,9.369540e+06,1.276980e+07,1.269237e+07,1.207279e+07,NaN,1.204738e+07,...,-2991206.0,-6.587060e+05,-1076625.0,2.091091e+06,2392935.0,4.037520e+05,NaN,NaN,NaN,NaN
4,NaN,AEGISLOG.BO,ttm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
date = datetime.today().strftime('%Y-%m-%d')
writer = pd.ExcelWriter('Yahoo-Finance-Scrape-' + date + '.xlsx')
df_combined.to_excel(writer)
writer.save()